In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests as req
import os
import matplotlib.pyplot as plt
print("Imports loaded!")

Imports loaded!


In [4]:
import datetime
test_adress = "https://www.morizon.pl/mieszkania/warszawa/wlochy/?ps%5Blocation%5D%5Btext_queue%5D%5B0%5D=Warszawa%20W%C5%82ochy&ps%5Blocation%5D%5Btext_queue%5D%5B1%5D=Warszawa%20Ursus&ps%5Blocation%5D%5Btext_queue%5D%5B2%5D=Warszawa%20Bemowo&ps%5Blocation%5D%5Bmap%5D=1&ps%5Blocation%5D%5Bmap_bounds%5D=52.280490899999,20.998632463328:52.140518099999,20.842100236674&ps%5Bnumber_of_rooms_from%5D=2&ps%5Bnumber_of_rooms_to%5D=3&ps%5Bwith_price%5D=1"
links = []
adresses = []
prices = []
cities = []
districts = []
streets = []
meters_array = []
room_numbers = []
levels = []
readed_prices_per_meter = []
def scrap_single_page(morizon_url):
    text_of_website = req.get(morizon_url).text
    soup = BeautifulSoup(text_of_website)
    flats_tiles = soup.find_all("div", {"class", "card__outer"})
    for flat in flats_tiles:
        try:
            flat_link = "-"
            flat_link = flat.find("a")["href"]
            
            basic_location_data = flat.find("span", {"class", "HNFyvY"}).text
            city = "-"
            city = basic_location_data.split("•")[0].strip()
            district = "-"
            district = basic_location_data.split("•")[1].strip()
            street = "-"
            street = flat.find("span", {"class", "na3ky-"}).text
            property_info_text = flat.find("div", {"class", "property-info"}).text
            readed_meters = 0
            readed_meters = float(property_info_text.split()[0])
            rooms = 0
            rooms = int(property_info_text.split("•")[1].replace(" pokoje", ""))
            floor = 0
            floor = property_info_text.split("•")[2].replace("piętro ", "").replace("parter", "0")
            flat_price = 0
            flat_price = int(flat.find("div", {"class", "Fzi-XT"}).text.replace(" ", "").replace("zł", ""))
            
            prices.append(flat_price)
            cities.append(city)
            districts.append(district)
            streets.append(street)
            room_numbers.append(rooms)
            meters_array.append(readed_meters)
            levels.append(floor)
            links.append(flat_link)
        except:
            print("Jednego z mieszkań nie udało się zbadać.")

def scrap_many_pages(starting_adress, max_page=1):
    scrap_single_page(starting_adress)
    if max_page > 1:
        for i in range(2, max_page+1):
            scrap_single_page(starting_adress + "&page={}".format(i))

    returned_df = pd.DataFrame.from_dict({"link": links, "City": cities, "District": districts, "Street": streets, "Meters": meters_array, "Floor": levels, "Rooms": room_numbers, "Price": prices})
    return returned_df
readed_flats = scrap_many_pages(test_adress, 100)


Jednego z mieszkań nie udało się zbadać.
Jednego z mieszkań nie udało się zbadać.
Jednego z mieszkań nie udało się zbadać.
Jednego z mieszkań nie udało się zbadać.


In [ ]:
folder_dir = r"path_to_data_folder"

def write_df_with_timestamp(flats_df):
    now = datetime.datetime.now()
    timestamp = now.strftime("%d_%m_%Y_%H_%M")
    filename = f"flats_{timestamp}.csv"
    flats_df.to_csv(os.path.join(folder_dir, filename))
write_df_with_timestamp(readed_flats)